In [11]:
# standard libraries import
import os
import glob
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Any
from tqdm import tqdm

import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import shapely.ops as so

## reading as df-like format using geopandas library
import geopandas as gpd

In [4]:
rootDir = Path('metadata')
floor_map = {"B2": -2, "B1": -1, "F1": 0, "F2": 1, "F3": 2, "F4": 3, "F5": 4, "F6": 5, "F7": 6, "F8": 7, "F9": 8,
             "1F": 0, "2F": 1, "3F": 2, "4F": 3, "5F": 4, "6F": 5, "7F": 6, "8F": 7, "9F": 8}
sampleCsvPath = 'sample_submission.csv'
imageOutputDir = 'referencePublicNotebooks/floorDataImages'
compareImgFolder = 'referencePublicNotebooks/compareImage'
##proj = pyproj.Transformer.from_crs(3857, 4326, always_xy=True)
maxFloorDimension = 440.0
maxFigureSize = 22.0

In [5]:
def generate_target_buildings() -> List[str]:
    ssubm = pd.read_csv(sampleCsvPath)
    ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
    buildingsList = sorted(ssubm_df[0].value_counts().index.tolist()) # type: ignore
    return buildingsList

In [6]:
def getFloorInfoJson(jsonFilePath):
    # Opening JSON file 
    f = open(jsonFilePath)
    # returns JSON object as  a dictionary 
    floorInfo = json.load(f) 
    return np.array([floorInfo['map_info']['width'],floorInfo['map_info']['height']])

def getFigureSize(floorInfo):
    figureSize = np.round((floorInfo/maxFloorDimension)  * maxFigureSize)
    return figureSize

In [7]:
def generateImgPaths():
    originalImgPaths = []
    generatedImgPaths = []

    for index,building in enumerate(buildingsList):    
        building = buildingsList[index]
        building_path = rootDir / building
        folders = sorted(building_path.glob('*'))

        for i,folder in enumerate(folders):
            originalImgPaths.append(str(sorted(folder.glob("*_image.png"))[0]))
            generatedImgPaths.append(f"{imageOutputDir}/{index}_{building}_{folder.name}.png")
    return originalImgPaths, generatedImgPaths

In [8]:
buildingsList = generate_target_buildings()

```python
for index,building in enumerate(buildingsList):    
    building = buildingsList[index]
    building_path = rootDir / building
    folders = sorted(building_path.glob('*'))
    print(f"{index} bdg - {building}")

    for i,folder in enumerate(folders):
        print(folder)
        ## geoJson boundary points
        geoJsonPath = sorted(folder.glob("*_map.json"))[0]
        with open(geoJsonPath, 'rb') as inputFile:
            geofloor_data = json.load(inputFile)

        ## scale of image
        floorDataJsonPath = sorted(folder.glob("*_info.json"))[0]
        floorInfo = getFloorInfoJson(floorDataJsonPath)
        figureSize = getFigureSize(floorInfo)

        type_poly = geofloor_data['features'][0]['geometry']['type']
        if type_poly == 'Polygon':
            polygon = np.array(geofloor_data['features'][0]['geometry']['coordinates'][0])
        else:
            polygon = np.array(geofloor_data['features'][0]['geometry']['coordinates'][0][0])

        floor_polygons = Polygon(polygon)
        store_polygons_l = [Polygon(features['geometry']['coordinates'][0]) for features in geofloor_data['features'][1:]]
        store_polygons = so.unary_union(store_polygons_l)

        ## plot output
        plt.ioff()
        plt.figure(figsize=(figureSize[0],figureSize[1]), frameon=False)

        ## plot floor polygons
        xs, ys = floor_polygons.exterior.coords.xy 
        plt.plot(xs, ys, 'k')        
        
        ## plot store boundaries
        if isinstance(store_polygons, Polygon):
            xs, ys = store_polygons.exterior.coords.xy 
            plt.plot(xs, ys, 'k')        
        else:
            for geom in store_polygons.geoms:    
                xs, ys = geom.exterior.xy   
                plt.plot(xs, ys, 'k')        
        
        plt.axis('off')
        plt.savefig(f"{imageOutputDir}/{index}_{building}_{folder.name}.png", dpi=300, bbox_inches='tight', pad_inches=0);
        plt.close()
```             

In [9]:
originalImgPaths, generatedImgPaths = generateImgPaths()

In [17]:
plt.ioff()

for i in tqdm(range(len(originalImgPaths))):
    ## subplot to plot
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
    ## read image
    origImg = cv2.imread(originalImgPaths[i])
    ## origImg = cv2.cvtColor(origImg, cv2.COLOR_BGR2GRAY)
    genImg  = cv2.imread(generatedImgPaths[i], cv2.IMREAD_GRAYSCALE)

    ## get info from path
    buildingName = generatedImgPaths[i].split("_")[1]
    floorName    = generatedImgPaths[i].split("_")[-1].split(".")[0]

    ## plot output
    axes[0].imshow(origImg)
    axes[0].set_title(f"{buildingName}_{floorName}_orig")
    axes[0].grid(True)

    axes[1].imshow(genImg, cmap=cm.gray)
    axes[1].set_title(f"{buildingName}_{floorName}_gen")
    axes[1].grid(True)
    
    plt.axis('off')
    plt.savefig(f"{compareImgFolder}/{buildingName}_{floorName}.png", dpi=200, bbox_inches='tight', pad_inches=0);
    plt.close()

100%|██████████| 139/139 [03:46<00:00,  1.63s/it]


img = np.expand_dims(cv2.imread('5a0546857ecc773753327266_B1.png', cv2.IMREAD_GRAYSCALE), axis=0)
temp = np.repeat(img,3,axis=0)
print(temp.dtype, temp.shape)